In [1]:
%load_ext autoreload
%autoreload 2

# Read train and Validation data

In [2]:
# import resources for handling dataset
from utils.LIPDataset import LIPDataset, RandomCrop, RandomHorizontalFlip, Resize, plot_data, ToTensor
from torchvision import transforms
import torch




In [3]:
# read dataset and provide augmentation
# TRAIN DATA
train_transform = transforms.Compose([
                Resize(256),
                RandomCrop(224),
                RandomHorizontalFlip(),
                ToTensor()
            ])

train_data = LIPDataset('dataset/train_data.ftr', 'dataset/train_set', train_transform)

In [4]:
# resource to display from dataset
from utils.LIPDataset import untransform_n_display

In [5]:
# visualize a sample
untransform_n_display(train_data, 5000)

In [6]:
# VALIDATION DATA
val_transform = transforms.Compose([
                        Resize(224),
                        ToTensor()
                        ])

val_data = LIPDataset('dataset/val_data.ftr', 'dataset/val_set', val_transform)

In [7]:
# visualize few sample
untransform_n_display(val_data, 6001)

In [8]:
# length of both
len(train_data), len(val_data), train_data[5]['image'].size(), train_data[6]['keypoints'].size()

(30462, 10000, torch.Size([3, 224, 224]), torch.Size([32]))

## Dataloader and batch size

In [9]:
from torch.utils.data import DataLoader

In [10]:
batch_size = 8

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4)

valid_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True, num_workers=4)

In [11]:
samples = next(iter(train_loader))

In [12]:
samples['keypoints'].size() # batches are formed

torch.Size([8, 32])

# Load model. Define lose function and optimizer

In [2]:
from utils.TrainModel import initialize_model

num_keypoints = 16
model = initialize_model(2*num_keypoints) 

In [3]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), strid

In [13]:
# testing a forward pass
# model = model.cuda()
# pred_keypts = model.forward(samples['image'].cuda())